In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/zstikg/NewsConcept Data-set.json
/kaggle/input/zstikg/DVD playerData-set.json
/kaggle/input/zstikg/MedicalConcept Data-set.json
/kaggle/input/zstikg/Cellular phone Data-set.json
/kaggle/input/zstikg/Digital camera2 Data-set.json
/kaggle/input/zstikg/Mp3 playerData-set.json
/kaggle/input/zstikg/Digital camera1 Data-set.json
/kaggle/input/keywords/Keyword_Medical.json
/kaggle/input/keywords/Keyword_Canon.json
/kaggle/input/keywords/Keyword_Creative.json
/kaggle/input/keywords/Keyword_Apex.json
/kaggle/input/keywords/Keyword_Apex 1.json
/kaggle/input/keywords/Keyword_Nokia.json
/kaggle/input/keywords/Keyword_News.json
/kaggle/input/keywords/Keyword_Nikon.json


In [2]:
# ============================================================================
# PART 1: INSTALLATION AND SETUP
# ============================================================================

# Install required packages
!pip install dspy-ai huggingface_hub networkx sentence-transformers pandas --quiet

print("✓ Packages installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [3]:
# ============================================================================
# PART 1: IMPORTS
# ============================================================================

import json
import networkx as nx
from typing import List, Dict, Set, Tuple
import dspy
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import defaultdict
import pandas as pd
import os

print("✓ All imports successful!")

2025-10-27 21:38:35.476005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761601115.666633      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761601115.723940      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✓ All imports successful!


In [4]:
# ============================================================================
# STEP 1: INSTALL AND IMPORT (if not done already)
# ============================================================================

import os
from huggingface_hub import InferenceClient
import dspy
import json
from typing import List, Tuple, Dict

print("✓ Imports complete")

✓ Imports complete


In [5]:
# ============================================================================
# STEP 2: CONFIGURE HUGGING FACE WITH DSPY.LM (PROPER WAY)
# ============================================================================
import os
import dspy

os.environ['HUGGINGFACE_API_KEY'] = 'KEY'

# Use DSPy's built-in LM class with the correct prefix
lm = dspy.LM(
    model='huggingface/meta-llama/Llama-3.1-8B-Instruct',
    api_key=os.environ['HUGGINGFACE_API_KEY'],
    max_tokens=12000,
    temperature=0.3
)

dspy.settings.configure(lm=lm)
print("✓ Llama-3.1-8B-Instruct configured successfully!")

✓ Llama-3.1-8B-Instruct configured successfully!


In [6]:
# ============================================================================
# PART 1: DATA LOADING FUNCTION
# ============================================================================

def load_json_data(filepath: str) -> List[Dict]:
    """
    Load JSON data from file
    
    Expected format:
    [
        {
            "Article Title": [],
            "Article Text": "text here...",
            "Concept": ["concept1", "concept2"]
        },
        ...
    ]
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"✓ Loaded {len(data)} documents from {filepath}")
    return data

# Test with sample data
sample_data = [
    {
        "Article Title": [],
        "Article Text": "excellent phone, excellent service.",
        "Concept": []
    },
    {
        "Article Title": [],
        "Article Text": "i am a business user who heavily depend on mobile service.",
        "Concept": ["service"]
    }
]

print("✓ Sample data ready for testing")
print(f"  Sample has {len(sample_data)} documents")

✓ Sample data ready for testing
  Sample has 2 documents


In [7]:
# ============================================================================
# FIND THE DATASET FILES
# ============================================================================

import os

# List files in your dataset directory
dataset_path = '/kaggle/input/zstikg'

print("Files in the dataset:")
print("=" * 60)

try:
    files = os.listdir(dataset_path)
    for i, file in enumerate(files, 1):
        print(f"{i}. {file}")
        full_path = os.path.join(dataset_path, file)
        print(f"   Full path: {full_path}")
        print()
except Exception as e:
    print(f"Error: {e}")
    print("\nTrying to list all datasets...")
    for dataset in os.listdir('/kaggle/input/'):
        print(f"- {dataset}")



Files in the dataset:
1. NewsConcept Data-set.json
   Full path: /kaggle/input/zstikg/NewsConcept Data-set.json

2. DVD playerData-set.json
   Full path: /kaggle/input/zstikg/DVD playerData-set.json

3. MedicalConcept Data-set.json
   Full path: /kaggle/input/zstikg/MedicalConcept Data-set.json

4. Cellular phone Data-set.json
   Full path: /kaggle/input/zstikg/Cellular phone Data-set.json

5. Digital camera2 Data-set.json
   Full path: /kaggle/input/zstikg/Digital camera2 Data-set.json

6. Mp3 playerData-set.json
   Full path: /kaggle/input/zstikg/Mp3 playerData-set.json

7. Digital camera1 Data-set.json
   Full path: /kaggle/input/zstikg/Digital camera1 Data-set.json



In [8]:
# ============================================================================
# LOAD ONE SPECIFIC DATASET
# ============================================================================

# Choose which dataset you want to work with:
# Option 1: Cellular phone
filepath = '/kaggle/input/zstikg/MedicalConcept Data-set.json'

# Option 2: News
# filepath = '/kaggle/input/zsltikg/NewsConcept Data-set.json'

# Option 3: Medical
# filepath = '/kaggle/input/zsltikg/MedicalConcept Data-set.json'

# Load the data
data = load_json_data(filepath)[:100]

print(f"✓ Loaded {len(data)} documents")
print(f"\nFirst document preview:")
print(f"  Keys: {list(data[0].keys())}")
print(f"  Text: {data[0].get('Article Text', '')[:100]}...")
print(f"  Concepts: {data[0].get('Concept', [])}")

✓ Loaded 2066 documents from /kaggle/input/zstikg/MedicalConcept Data-set.json
✓ Loaded 100 documents

First document preview:
  Keys: ['Article Title', 'Article Text', 'Concept']
  Text: 1. Health care reform

How could the health care reform legislation that President Barack Obama sign...
  Concepts: []


In [9]:
# ============================================================================
# PROCESS YOUR ALL THE ARTICLES
# ============================================================================

print("=" * 80)
print("PROCESSING ALL THE ARTICLES")
print("=" * 80)

# data is now correctly loaded as a list of dicts
individual_articles = []

for idx, item in enumerate(data):
    article = {
        'id': idx,
        'Article Text': item['Article Text'],
        'Concept': item['Concept'] if item['Concept'] else []
    }
    individual_articles.append(article)

print(f"\n✓ Created list of {len(individual_articles)} individual articles")

# Show first 3
print(f"\nFirst 3 articles:")
print("-" * 80)
for i in range(min(3, len(individual_articles))):
    article = individual_articles[i]
    print(f"\nArticle {article['id']}:")
    print(f"  Text: {article['Article Text'][:80]}...")
    print(f"  Concepts: {article['Concept']}")

PROCESSING ALL THE ARTICLES

✓ Created list of 100 individual articles

First 3 articles:
--------------------------------------------------------------------------------

Article 0:
  Text: 1. Health care reform

How could the health care reform legislation that Preside...
  Concepts: []

Article 1:
  Text: My colleagues at Harvard Health Publishing and I have a mission: to provide accu...
  Concepts: ['Heart Health', "Women's Health"]

Article 2:
  Text: Does echinacea, the popular natural cold remedy, really work?

It depends on wha...
  Concepts: []


In [10]:
# ============================================================================
# STEP 3: DEFINE SIGNATURES
# ============================================================================

class EntityExtractor(dspy.Signature):
    """Extract key entities from the given text. Extracted entities are nouns, 
    verbs, or adjectives, particularly regarding sentiment. This is for an 
    extraction task, please be thorough and accurate to the reference text.
    
    Return ONLY a valid JSON list format: ["entity1", "entity2", "entity3"]
    """
    
    text = dspy.InputField(desc="The text to extract entities from")
    entities = dspy.OutputField(desc="List of extracted entities in JSON format")

class RelationExtractor(dspy.Signature):
    """Extract subject-predicate-object triples from the assistant message. 
    A predicate (1-3 words) defines the relationship between the subject and 
    object. Relationship may be fact or sentiment based on assistant's message. 
    Subject and object are entities. Entities provided are from the assistant 
    message and prior conversation history, though you may not need all of them. 
    This is for an extraction task, please be thorough, accurate, and faithful 
    to the reference text.
    
    Return ONLY valid JSON format: [["subject1", "predicate1", "object1"], ["subject2", "predicate2", "object2"]]
    """
    
    text = dspy.InputField(desc="The text to extract relations from")
    entities = dspy.InputField(desc="List of available entities")
    triples = dspy.OutputField(desc="List of [subject, predicate, object] triples in JSON format")

print("✓ Signatures defined")

✓ Signatures defined


In [11]:
# ============================================================================
# STEP 4: CREATE ENTITY EXTRACTOR
# ============================================================================

class ExtractEntities(dspy.Module):
    def __init__(self):
        super().__init__()
        self.extract = dspy.ChainOfThought(EntityExtractor)
    
    def forward(self, text: str) -> List[str]:
        if not text or len(text.strip()) < 3:
            return []
            
        result = self.extract(text=text)
        
        try:
            entities_text = result.entities.strip()
            
            if '[' in entities_text and ']' in entities_text:
                start = entities_text.find('[')
                end = entities_text.rfind(']') + 1
                entities_text = entities_text[start:end]
            
            entities = json.loads(entities_text)
            
            if isinstance(entities, list):
                return [str(e).lower().strip() for e in entities if e and len(str(e).strip()) > 1]
            return []
            
        except:
            try:
                entities_text = result.entities.strip()
                if entities_text.startswith('['):
                    entities_text = entities_text[1:]
                if entities_text.endswith(']'):
                    entities_text = entities_text[:-1]
                
                entities = []
                for item in entities_text.split(','):
                    item = item.strip(' "\'\n\t')
                    if item and len(item) > 1:
                        entities.append(item.lower())
                
                return entities[:50]
            except:
                return []

entity_extractor = ExtractEntities()
print("✓ Entity Extractor created")

✓ Entity Extractor created


In [12]:
# ============================================================================
# STEP 5: CREATE RELATION EXTRACTOR
# ============================================================================

class ExtractRelations(dspy.Module):
    def __init__(self):
        super().__init__()
        self.extract = dspy.ChainOfThought(RelationExtractor)
    
    def forward(self, text: str, entities: List[str]) -> List[Tuple[str, str, str]]:
        if not entities or not text:
            return []
        
        entities_subset = entities[:30]
        entities_str = json.dumps(entities_subset)
        
        result = self.extract(text=text, entities=entities_str)
        
        try:
            triples_text = result.triples.strip()
            
            if '[' in triples_text and ']' in triples_text:
                start = triples_text.find('[')
                end = triples_text.rfind(']') + 1
                triples_text = triples_text[start:end]
            
            triples = json.loads(triples_text)
            
            normalized_triples = []
            for triple in triples:
                if isinstance(triple, (list, tuple)) and len(triple) == 3:
                    s, p, o = triple
                    s = str(s).lower().strip()
                    p = str(p).lower().strip()
                    o = str(o).lower().strip()
                    
                    if s and p and o and s != o:
                        normalized_triples.append((s, p, o))
            
            return normalized_triples[:100]
            
        except Exception as e:
            return []

relation_extractor = ExtractRelations()
print("✓ Relation Extractor created")

✓ Relation Extractor created


In [13]:
# ============================================================================
# STEP 6: TEST EXTRACTION
# ============================================================================

print("\nTesting Entity & Relation Extraction...")
print("=" * 80)

# Test on first 2 documents
for i in range(min(2, len(data))):
    text = data[i].get('Article Text', '')
    
    print(f"\nDocument {i+1}:")
    print(f"  Text: {text[:80]}...")
    
    # Extract entities
    print(f"  Extracting entities...")
    entities = entity_extractor(text)
    print(f"  ✓ Found {len(entities)} entities: {entities[:5]}...")
    
    # Extract relations
    if entities:
        print(f"  Extracting relations...")
        relations = relation_extractor(text, entities)
        print(f"  ✓ Found {len(relations)} relations")
        
        if relations:
            for j, (s, p, o) in enumerate(relations[:3], 1):
                print(f"    {j}. ({s}) --[{p}]--> ({o})")

print("\n" + "=" * 80)
print("✓ Test complete!")


Testing Entity & Relation Extraction...

Document 1:
  Text: 1. Health care reform

How could the health care reform legislation that Preside...
  Extracting entities...
  ✓ Found 39 entities: ['health care reform', 'president barack obama', 'patient protection and affordable care act', 'american heart association', 'cardiopulmonary resuscitation']...
  Extracting relations...
  ✓ Found 16 relations
    1. (president barack obama) --[signed into law]--> (patient protection and affordable care act)
    2. (patient protection and affordable care act) --[extends coverage to]--> (32 million americans)
    3. (patient protection and affordable care act) --[reins in health care spending]--> (by 2019)

Document 2:
  Text: My colleagues at Harvard Health Publishing and I have a mission: to provide accu...
  Extracting entities...
  ✓ Found 60 entities: ['harvard health publishing', 'health', 'lives', 'mission', 'feedback']...
  Extracting relations...
  ✓ Found 15 relations
    1. (harvard he

In [14]:
# ============================================================================
# CLUSTERING SIGNATURES (DEFINE FIRST!)
# ============================================================================

class ClusterValidator(dspy.Signature):
    """Verify if these entities belong in the same cluster.
    A cluster should contain entities that are the same in meaning, with different:
    - tenses, plural forms, stem forms, upper/lower cases
    Or entities with close semantic meanings.
    
    Return ONLY valid JSON format: ["entity1", "entity2", "entity3"]
    Return only entities you are confident belong together.
    If not confident, return empty list [].
    """
    
    entities = dspy.InputField(desc="Entities to validate")
    valid_cluster = dspy.OutputField(desc="Validated cluster in JSON format")

print("✓ ClusterValidator Signature defined")

✓ ClusterValidator Signature defined


In [15]:
# ============================================================================
# SEMANTIC SIMILARITY CLUSTERING (FROM PAPER)
# ============================================================================

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class SemanticEntityClustering(dspy.Module):
    def __init__(self, similarity_threshold=0.75):
        super().__init__()
        self.validator = dspy.ChainOfThought(ClusterValidator)
        
        # Load embedding model (same as paper)
        print("Loading sentence transformer model...")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        print("✓ Model loaded")
        
        self.similarity_threshold = similarity_threshold
    
    def _parse_cluster(self, text: str) -> List[str]:
        """Parse cluster from LLM response"""
        try:
            text = text.strip()
            if '[' in text and ']' in text:
                start = text.find('[')
                end = text.rfind(']') + 1
                text = text[start:end]
            
            cluster = json.loads(text)
            if isinstance(cluster, list):
                return [str(e).lower().strip() for e in cluster if e]
            return []
        except:
            return []
    
    def _get_semantic_clusters(self, entities: List[str]) -> List[List[str]]:
        """Group entities by semantic similarity using embeddings"""
        
        if len(entities) == 0:
            return []
        
        # Get embeddings for all entities
        embeddings = self.model.encode(entities)
        
        # Compute pairwise cosine similarity
        similarity_matrix = cosine_similarity(embeddings)
        
        # Find clusters using similarity threshold
        clusters = []
        remaining = set(range(len(entities)))
        
        for i in range(len(entities)):
            if i not in remaining:
                continue
            
            # Find all entities similar to this one
            cluster_indices = [i]
            remaining.discard(i)
            
            for j in range(i + 1, len(entities)):
                if j not in remaining:
                    continue
                
                # Check if similar enough
                if similarity_matrix[i][j] >= self.similarity_threshold:
                    cluster_indices.append(j)
                    remaining.discard(j)
            
            # Convert indices to entity names
            cluster = [entities[idx] for idx in cluster_indices]
            
            # Only keep clusters with 2-4 entities
            if 2 <= len(cluster) <= 4:
                clusters.append(cluster)
            elif len(cluster) == 1:
                # Keep singletons for later
                pass
        
        return clusters
    
    def forward(self, entities: List[str]) -> Dict[str, List[str]]:
        """Semantic clustering with LLM validation"""
        
        print(f"Starting semantic clustering with {len(entities)} entities...")
        print(f"  Similarity threshold: {self.similarity_threshold}")
        
        # Remove duplicates
        unique_entities = list(set(entities))
        
        # Step 1: Find semantic clusters using embeddings
        print("  Computing semantic similarities...")
        potential_clusters = self._get_semantic_clusters(unique_entities)
        
        print(f"  Found {len(potential_clusters)} potential clusters")
        
        # Step 2: Validate with LLM
        validated_clusters = {}
        cluster_id = 0
        clustered_entities = set()
        
        for cluster in potential_clusters:
            try:
                # Ask LLM to validate
                validation = self.validator(entities=json.dumps(cluster))
                validated = self._parse_cluster(validation.valid_cluster)
                
                if validated and len(validated) >= 2:
                    cluster_label = validated[0]
                    validated_clusters[cluster_label] = validated
                    
                    for entity in validated:
                        clustered_entities.add(entity)
                    
                    print(f"  ✓ Cluster {cluster_id}: {validated}")
                    cluster_id += 1
                else:
                    # LLM rejected - add as singletons
                    for entity in cluster:
                        if entity not in clustered_entities:
                            validated_clusters[entity] = [entity]
                            clustered_entities.add(entity)
            except:
                # Error - add as singletons
                for entity in cluster:
                    if entity not in clustered_entities:
                        validated_clusters[entity] = [entity]
                        clustered_entities.add(entity)
        
        # Step 3: Add all remaining entities as singletons
        for entity in unique_entities:
            if entity not in clustered_entities:
                validated_clusters[entity] = [entity]
        
        multi = sum(1 for v in validated_clusters.values() if len(v) > 1)
        print(f"✓ Semantic clustering complete: {len(validated_clusters)} total clusters")
        print(f"  Multi-entity clusters: {multi}")
        print(f"  Singleton entities: {len(validated_clusters) - multi}")
        
        return validated_clusters

# Create semantic clusterer with different thresholds
entity_clusterer_semantic = SemanticEntityClustering(similarity_threshold=0.75)
print("\n✓ Semantic Entity Clustering Module created")

Loading sentence transformer model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Model loaded

✓ Semantic Entity Clustering Module created


In [16]:
# ============================================================================
# TEST SEMANTIC CLUSTERING
# ============================================================================

print("\nTesting SEMANTIC Clustering (Embeddings + LLM)...")
print("=" * 80)

test_entities = [
    # Should cluster (same concept)
    'phone', 'mobile phone', 'cellphone', 'cell phone',
    
    # Should cluster (same concept)
    'usa', 'united states', 'america',
    
    # Should cluster (similar meaning)
    'helping hand', 'assistant', 'helper', 'support',
    
    # Should cluster (same concept)
    'service', 'services',
    
    # Should cluster (same concept)
    'call', 'calling', 'calls',
    
    # Should NOT cluster (different brands)
    'nokia', 'motorola', 'samsung',
    
    # Should NOT cluster (different concepts)
    'signal', 'battery', 'screen',
]

print(f"Testing with {len(test_entities)} entities:")
for e in test_entities:
    print(f"  - {e}")

print("\n" + "=" * 80)

# Cluster with semantic method
semantic_clusters = entity_clusterer_semantic(test_entities)

print("\n" + "=" * 80)
print("SEMANTIC CLUSTERING RESULTS:")
print("=" * 80)

# Show multi-entity clusters
multi_clusters = {k: v for k, v in semantic_clusters.items() if len(v) > 1}

if multi_clusters:
    print(f"\n✓ Found {len(multi_clusters)} semantic clusters:")
    for i, (label, members) in enumerate(multi_clusters.items(), 1):
        print(f"  {i}. {members}")
        
        # Show similarity scores
        if len(members) > 1:
            embeddings = entity_clusterer_semantic.model.encode(members)
            similarities = cosine_similarity(embeddings)
            avg_sim = (similarities.sum() - len(members)) / (len(members) * (len(members) - 1))
            print(f"     Average similarity: {avg_sim:.3f}")
else:
    print("\n  No clusters found")

print(f"\nSingleton entities: {len(semantic_clusters) - len(multi_clusters)}")

print("\n✓ Test complete!")


Testing SEMANTIC Clustering (Embeddings + LLM)...
Testing with 22 entities:
  - phone
  - mobile phone
  - cellphone
  - cell phone
  - usa
  - united states
  - america
  - helping hand
  - assistant
  - helper
  - support
  - service
  - services
  - call
  - calling
  - calls
  - nokia
  - motorola
  - samsung
  - signal
  - battery
  - screen

Starting semantic clustering with 22 entities...
  Similarity threshold: 0.75
  Computing semantic similarities...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  Found 3 potential clusters
  ✓ Cluster 0: ['services', 'service']
  ✓ Cluster 1: ['call', 'calls', 'calling']
  ✓ Cluster 2: ['united states', 'america', 'usa']
✓ Semantic clustering complete: 17 total clusters
  Multi-entity clusters: 3
  Singleton entities: 14

SEMANTIC CLUSTERING RESULTS:

✓ Found 3 semantic clusters:
  1. ['services', 'service']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

     Average similarity: 0.921
  2. ['call', 'calls', 'calling']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

     Average similarity: 0.878
  3. ['united states', 'america', 'usa']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

     Average similarity: 0.844

Singleton entities: 14

✓ Test complete!


In [17]:
# ============================================================================
# VISUALIZE HOW CLUSTERING WORKS
# ============================================================================

from sklearn.metrics.pairwise import cosine_similarity

# Test entities
test_entities = ['phone', 'mobile phone', 'cellphone', 'nokia', 'motorola']

# Get embeddings
embeddings = entity_clusterer_semantic.model.encode(test_entities)

# Calculate similarities
print("Similarity Matrix:")
print("=" * 80)
print(f"{'':15s}", end='')
for e in test_entities:
    print(f"{e:15s}", end='')
print()

for i, entity1 in enumerate(test_entities):
    print(f"{entity1:15s}", end='')
    for j, entity2 in enumerate(test_entities):
        similarity = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
        print(f"{similarity:15.3f}", end='')
    print()

print("\n" + "=" * 80)
print(f"Threshold: 0.75")
print("=" * 80)

# Show which pairs would cluster
print("\nPairs above threshold (would cluster together):")
for i, entity1 in enumerate(test_entities):
    for j, entity2 in enumerate(test_entities):
        if i < j:  # Avoid duplicates
            similarity = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
            if similarity >= 0.75:
                print(f"  ✓ {entity1:15s} ↔ {entity2:15s} = {similarity:.3f}")

print("\nPairs below threshold (stay separate):")
for i, entity1 in enumerate(test_entities):
    for j, entity2 in enumerate(test_entities):
        if i < j:
            similarity = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
            if similarity < 0.75:
                print(f"  ✗ {entity1:15s} ↔ {entity2:15s} = {similarity:.3f}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity Matrix:
               phone          mobile phone   cellphone      nokia          motorola       
phone                    1.000          0.814          0.845          0.681          0.644
mobile phone             0.814          1.000          0.843          0.751          0.688
cellphone                0.845          0.843          1.000          0.674          0.617
nokia                    0.681          0.751          0.674          1.000          0.640
motorola                 0.644          0.688          0.617          0.640          1.000

Threshold: 0.75

Pairs above threshold (would cluster together):
  ✓ phone           ↔ mobile phone    = 0.814
  ✓ phone           ↔ cellphone       = 0.845
  ✓ mobile phone    ↔ cellphone       = 0.843
  ✓ mobile phone    ↔ nokia           = 0.751

Pairs below threshold (stay separate):
  ✗ phone           ↔ nokia           = 0.681
  ✗ phone           ↔ motorola        = 0.644
  ✗ mobile phone    ↔ motorola        = 0.688
  ✗ cel

In [18]:
# ============================================================================
# FINAL KGGEN WITH SEMANTIC CLUSTERING
# ============================================================================

class KGGenSemantic:
    def __init__(self, similarity_threshold=0.75):
        self.entity_extractor = entity_extractor
        self.relation_extractor = relation_extractor
        self.entity_clusterer = SemanticEntityClustering(similarity_threshold=similarity_threshold)
        self.graph = nx.DiGraph()
        self.entity_clusters = {}
        
    def generate_from_json(self, json_data: List[Dict], max_docs: int = None) -> nx.DiGraph:
        """Generate KG from JSON dataset"""
        all_entities = set()
        all_relations = []
        
        if max_docs:
            json_data = json_data[:max_docs]
        
        print(f"Processing {len(json_data)} documents...")
        print("=" * 80)
        
        for idx, item in enumerate(json_data):
            text = item.get('Article Text', '')
            concepts = item.get('Concept', [])
            
            if not text or len(text.strip()) < 5:
                continue
            
            try:
                # Extract entities
                entities = self.entity_extractor(text)
                all_entities.update(entities)
                
                # Add concepts
                for concept in concepts:
                    if concept and isinstance(concept, str):
                        all_entities.add(concept.lower().strip())
                
                # Extract relations
                relations = self.relation_extractor(text, list(all_entities))
                all_relations.extend(relations)
                
                if (idx + 1) % 20 == 0:
                    print(f"  {idx + 1}/{len(json_data)} docs | {len(all_entities)} entities | {len(all_relations)} relations")
                    
            except Exception as e:
                continue
        
        print(f"\n✓ Extraction complete!")
        print(f"  Total entities: {len(all_entities)}")
        print(f"  Total relations: {len(all_relations)}")
        
        # Build graph
        for subj, pred, obj in all_relations:
            self.graph.add_edge(subj, obj, relation=pred)
        
        print(f"  Graph nodes: {len(self.graph.nodes())}")
        print(f"  Graph edges: {len(self.graph.edges())}")
        
        return self.graph
    
    def cluster_entities(self):
        """Semantic clustering with embeddings"""
        nodes = list(self.graph.nodes())
        
        if len(nodes) == 0:
            print("No nodes to cluster!")
            return self.graph
        
        print(f"\n{'='*80}")
        print(f"SEMANTIC CLUSTERING: {len(nodes)} ENTITIES")
        print(f"{'='*80}")
        
        self.entity_clusters = self.entity_clusterer(nodes)
        
        # Map entities
        entity_mapping = {}
        for cluster_label, cluster_entities in self.entity_clusters.items():
            for entity in cluster_entities:
                entity_mapping[entity] = cluster_label
        
        # Rebuild graph
        new_graph = nx.DiGraph()
        for u, v, data in self.graph.edges(data=True):
            new_u = entity_mapping.get(u, u)
            new_v = entity_mapping.get(v, v)
            relation = data.get('relation', 'related_to')
            
            if new_u == new_v:
                continue
            
            if not new_graph.has_edge(new_u, new_v):
                new_graph.add_edge(new_u, new_v, relation=relation)
        
        self.graph = new_graph
        
        print(f"\n✓ Clustering complete!")
        print(f"  Final nodes: {len(self.graph.nodes())}")
        print(f"  Final edges: {len(self.graph.edges())}")
        
        return self.graph
    
    def save_graph(self, filepath: str):
        data = nx.node_link_data(self.graph)
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)
        print(f"✓ Saved to {filepath}")
    
    def export_triples(self, filepath: str):
        triples = []
        for u, v, data in self.graph.edges(data=True):
            triples.append({
                'subject': u,
                'predicate': data.get('relation', 'related_to'),
                'object': v
            })
        import pandas as pd
        df = pd.DataFrame(triples)
        df.to_csv(filepath, index=False)
        print(f"✓ Exported to {filepath}")

# Initialize semantic KGGen (threshold 0.75 = balanced)
kg_gen_semantic = KGGenSemantic(similarity_threshold=0.75)
print("\n✓ KGGen with Semantic Clustering (Embeddings + LLM) created!")

Loading sentence transformer model...
✓ Model loaded

✓ KGGen with Semantic Clustering (Embeddings + LLM) created!


In [19]:
# ============================================================================
# GENERATE KG FOR ALL THE ARTICLES
# ============================================================================

print("\n" + "=" * 80)
print("GENERATING KNOWLEDGE GRAPHS FOR ALL THE ARTICLES")
print("=" * 80)

all_article_kgs = []

total = len(individual_articles)
print(f"\nProcessing {total} articles...\n")

for idx, article in enumerate(individual_articles[:100]):
    article_id = article['id']
    text = article['Article Text']
    concepts = article['Concept']
    
    # Skip empty articles
    if not text or len(text.strip()) < 5:
        all_article_kgs.append({
            'id': article_id,
            'text': text,
            'concepts': concepts,
            'graph': nx.DiGraph(),
            'entities': [],
            'relations': [],
            'num_nodes': 0,
            'num_edges': 0
        })
        continue
    
    try:
        # Extract entities for THIS article
        entities = entity_extractor(text)
        
        # Add original concepts as entities
        for concept in concepts:
            if concept and isinstance(concept, str):
                entities.append(concept.lower().strip())
        
        entities = list(set(entities))  # Remove duplicates
        
        # Extract relations for THIS article
        if entities:
            relations = relation_extractor(text, entities)
        else:
            relations = []
        
        # Build graph for THIS article
        graph = nx.DiGraph()
        for subj, pred, obj in relations:
            graph.add_edge(subj, obj, relation=pred)
        
        # Store everything for this article
        all_article_kgs.append({
            'id': article_id,
            'text': text,
            'concepts': concepts,
            'graph': graph,
            'entities': entities,
            'relations': relations,
            'num_nodes': len(graph.nodes()),
            'num_edges': len(graph.edges())
        })
        
        # Progress update
        if (idx + 1) % 50 == 0:
            print(f"✓ Processed {idx + 1}/{total} articles...")
        
    except Exception as e:
        print(f"✗ Article {article_id}: Error - {str(e)[:100]}")
        all_article_kgs.append({
            'id': article_id,
            'text': text,
            'concepts': concepts,
            'graph': nx.DiGraph(),
            'entities': [],
            'relations': [],
            'num_nodes': 0,
            'num_edges': 0
        })

print(f"\n" + "=" * 80)
print("✓ KNOWLEDGE GRAPH GENERATION COMPLETE!")
print("=" * 80)
print(f"Total articles processed: {len(all_article_kgs)}")
print(f"Articles with graphs: {sum(1 for kg in all_article_kgs if kg['num_edges'] > 0)}")
print(f"Articles without graphs: {sum(1 for kg in all_article_kgs if kg['num_edges'] == 0)}")

# Statistics
total_entities = sum(len(kg['entities']) for kg in all_article_kgs)
total_relations = sum(len(kg['relations']) for kg in all_article_kgs)

print(f"\nTotal entities extracted: {total_entities}")
print(f"Total relations extracted: {total_relations}")

with_graphs = [kg for kg in all_article_kgs if kg['num_edges'] > 0]
if with_graphs:
    avg_nodes = sum(kg['num_nodes'] for kg in with_graphs) / len(with_graphs)
    avg_edges = sum(kg['num_edges'] for kg in with_graphs) / len(with_graphs)
    print(f"\nAverage nodes per KG: {avg_nodes:.2f}")
    print(f"Average edges per KG: {avg_edges:.2f}")


GENERATING KNOWLEDGE GRAPHS FOR ALL THE ARTICLES

Processing 100 articles...

✓ Processed 50/100 articles...
✓ Processed 100/100 articles...

✓ KNOWLEDGE GRAPH GENERATION COMPLETE!
Total articles processed: 100
Articles with graphs: 100
Articles without graphs: 0

Total entities extracted: 2701
Total relations extracted: 1425

Average nodes per KG: 19.47
Average edges per KG: 13.52


In [20]:
# ============================================================================
# CHECK WHAT'S BEEN SUCCESSFULLY PROCESSED
# ============================================================================

print("=" * 80)
print("CHECKING SUCCESSFUL PROCESSING")
print("=" * 80)

# Count successes
successful = [kg for kg in all_article_kgs if kg['num_edges'] > 0]
failed = [kg for kg in all_article_kgs if kg['num_edges'] == 0]

print(f"\nTotal articles: {len(all_article_kgs)}")
print(f"✓ Successfully processed: {len(successful)}")
print(f"✗ Failed/Empty: {len(failed)}")

if successful:
    print(f"\nSuccessful articles with KGs:")
    for kg in successful[:10]:
        print(f"  Article {kg['id']}: {kg['num_nodes']} nodes, {kg['num_edges']} edges")
    
    # Export the successful ones
    print("\n" + "=" * 80)
    print("EXPORTING SUCCESSFUL RESULTS")
    print("=" * 80)
    
    import json
    import os
    
    # Create output directory
    os.makedirs('/kaggle/working/individual_kgs', exist_ok=True)
    
    # Export successful KGs
    for kg in successful:
        article_id = kg['id']
        
        graph_data = {
            'article_id': article_id,
            'text': kg['text'],
            'concepts': kg['concepts'],
            'entities': kg['entities'],
            'relations': [
                {'subject': s, 'predicate': p, 'object': o}
                for s, p, o in kg['relations']
            ],
            'graph': nx.node_link_data(kg['graph'])
        }
        
        filepath = f'/kaggle/working/individual_kgs/article_{article_id}.json'
        with open(filepath, 'w') as f:
            json.dump(graph_data, f, indent=2)
    
    print(f"✓ Saved {len(successful)} JSON files")
    
    # Export triples
    all_triples = []
    for kg in successful:
        for subj, pred, obj in kg['relations']:
            all_triples.append({
                'article_id': kg['id'],
                'subject': subj,
                'predicate': pred,
                'object': obj
            })
    
    triples_df = pd.DataFrame(all_triples)
    triples_df.to_csv('/kaggle/working/successful_triples.csv', index=False)
    print(f"✓ Saved {len(all_triples)} triples to successful_triples.csv")
    
    # Summary
    summary = []
    for kg in all_article_kgs:
        summary.append({
            'article_id': kg['id'],
            'text_preview': kg['text'][:100],
            'num_entities': len(kg['entities']),
            'num_relations': len(kg['relations']),
            'num_nodes': kg['num_nodes'],
            'num_edges': kg['num_edges'],
            'status': 'success' if kg['num_edges'] > 0 else 'failed'
        })
    
    summary_df = pd.DataFrame(summary)
    summary_df.to_csv('/kaggle/working/processing_summary.csv', index=False)
    print(f"✓ Saved summary")
    
    print("\n✓ DONE! Downloaded what we could process.")
else:
    print("\n✗ No successful KGs to export")

CHECKING SUCCESSFUL PROCESSING

Total articles: 100
✓ Successfully processed: 100
✗ Failed/Empty: 0

Successful articles with KGs:
  Article 0: 46 nodes, 31 edges
  Article 1: 20 nodes, 16 edges
  Article 2: 12 nodes, 7 edges
  Article 3: 27 nodes, 18 edges
  Article 4: 13 nodes, 10 edges
  Article 5: 17 nodes, 9 edges
  Article 6: 46 nodes, 24 edges
  Article 7: 18 nodes, 16 edges
  Article 8: 15 nodes, 13 edges
  Article 9: 36 nodes, 27 edges

EXPORTING SUCCESSFUL RESULTS
✓ Saved 100 JSON files
✓ Saved 1425 triples to successful_triples.csv
✓ Saved summary

✓ DONE! Downloaded what we could process.


/usr/local/lib/python3.11/dist-packages/networkx/readwrite/json_graph/node_link.py:145: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


In [21]:
# ============================================================================
# VIEW SUCCESSFUL RESULTS
# ============================================================================

import pandas as pd

print("=" * 80)
print("YOUR 46 SUCCESSFUL KNOWLEDGE GRAPHS")
print("=" * 80)

# Load summary
summary_df = pd.read_csv('/kaggle/working/processing_summary.csv')
successful_df = summary_df[summary_df['status'] == 'success'].copy()

print(f"\nTop 10 Largest KGs:")
print("-" * 80)
top_kgs = successful_df.nlargest(10, 'num_edges')
print(top_kgs[['article_id', 'num_nodes', 'num_edges', 'text_preview']].to_string(index=False))

# Load triples
triples_df = pd.read_csv('/kaggle/working/successful_triples.csv')

print(f"\n\nSample Triples:")
print("-" * 80)
print(triples_df.head(20).to_string(index=False))

print(f"\n\nStatistics:")
print("-" * 80)
print(f"Total successful articles: {len(successful_df)}")
print(f"Total triples extracted: {len(triples_df)}")
print(f"Avg edges per KG: {successful_df['num_edges'].mean():.2f}")
print(f"Max edges in a KG: {successful_df['num_edges'].max()}")

print("\n" + "=" * 80)
print("FILES READY TO DOWNLOAD:")
print("=" * 80)
print("  📁 /kaggle/working/individual_kgs/ (46 JSON files)")
print("  📄 /kaggle/working/successful_triples.csv")
print("  📄 /kaggle/working/processing_summary.csv")

YOUR 46 SUCCESSFUL KNOWLEDGE GRAPHS

Top 10 Largest KGs:
--------------------------------------------------------------------------------
 article_id  num_nodes  num_edges                                                                                           text_preview
         28         44         32   Reading medical journals is the main occupational hazard I face as editor of the Harvard Heart Lette
          0         46         31 1. Health care reform\n\nHow could the health care reform legislation that President Barack Obama sign
         10         46         31 1. Health care reform\n\nHow could the health care reform legislation that President Barack Obama sign
          9         36         27   The contents displayed within this public group(s), such as text, graphics, and other material ("Con
         19         36         27   The contents displayed within this public group(s), such as text, graphics, and other material ("Con
         26         38         27   On O

In [22]:
# ============================================================================
# VISUALIZE INDIVIDUAL ARTICLE KNOWLEDGE GRAPHS
# ============================================================================

!pip install pyvis --quiet

from pyvis.network import Network
import networkx as nx
import os

def visualize_article_kg(kg_data, output_file):
    """
    Create interactive HTML visualization for one article's KG
    """
    graph = kg_data['graph']
    article_id = kg_data['id']
    
    if len(graph.nodes()) == 0:
        print(f"  Article {article_id}: No graph to visualize")
        return None
    
    print(f"  Article {article_id}: {len(graph.nodes())} nodes, {len(graph.edges())} edges")
    
    # Create pyvis network
    net = Network(
        height='800px',
        width='100%',
        bgcolor='#ffffff',
        font_color='black',
        directed=True,
        notebook=False
    )
    
    # Set physics options
    net.set_options("""
    {
      "physics": {
        "enabled": true,
        "barnesHut": {
          "gravitationalConstant": -8000,
          "centralGravity": 0.3,
          "springLength": 95,
          "springConstant": 0.04
        },
        "minVelocity": 0.75
      },
      "nodes": {
        "font": {"size": 16}
      },
      "edges": {
        "font": {"size": 12},
        "arrows": {"to": {"enabled": true, "scaleFactor": 0.5}}
      }
    }
    """)
    
    # Add nodes with colors based on degree
    for node in graph.nodes():
        degree = graph.degree(node)
        size = 20 + degree * 5
        
        # Color based on degree
        if degree > 5:
            color = '#e74c3c'  # Red for highly connected
        elif degree > 2:
            color = '#3498db'  # Blue for medium
        else:
            color = '#95a5a6'  # Gray for low
        
        net.add_node(
            node, 
            label=node, 
            size=size, 
            title=f"{node}\nConnections: {degree}",
            color=color
        )
    
    # Add edges with labels
    for source, target, data in graph.edges(data=True):
        relation = data.get('relation', 'related_to')
        net.add_edge(source, target, label=relation, title=relation)
    
    # Save
    net.save_graph(output_file)
    
    return net

# Create visualizations directory
os.makedirs('/kaggle/working/kg_visualizations', exist_ok=True)

print("=" * 80)
print("CREATING INTERACTIVE VISUALIZATIONS")
print("=" * 80)

# Visualize all successful KGs
visualized = 0
for kg in all_article_kgs:
    if kg['num_edges'] > 0:  # Only visualize articles with graphs
        article_id = kg['id']
        output_file = f'/kaggle/working/kg_visualizations/article_{article_id}_graph.html'
        visualize_article_kg(kg, output_file)
        visualized += 1

print(f"\n✓ Created {visualized} interactive visualizations!")
print(f"  Location: /kaggle/working/kg_visualizations/")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.9 MB/s eta 0:00:00a 0:00:01
CREATING INTERACTIVE VISUALIZATIONS
  Article 0: 46 nodes, 31 edges
  Article 1: 20 nodes, 16 edges
  Article 2: 12 nodes, 7 edges
  Article 3: 27 nodes, 18 edges
  Article 4: 13 nodes, 10 edges
  Article 5: 17 nodes, 9 edges
  Article 6: 46 nodes, 24 edges
  Article 7: 18 nodes, 16 edges
  Article 8: 15 nodes, 13 edges
  Article 9: 36 nodes, 27 edges
  Article 10: 46 nodes, 31 edges
  Article 11: 20 nodes, 16 edges
  Article 12: 12 nodes, 7 edges
  Article 13: 27 nodes, 18 edges
  Article 14: 13 nodes, 10 edges
  Article 15: 17 nodes, 9 edges
  Article 16: 46 nodes, 24 edges
  Article 17: 18 nodes, 16 edges
  Article 18: 15 nodes, 13 edges
  Article 19: 36 nodes, 27 edges
  Article 20: 27 nodes, 19 edges
  Article 21: 22 nodes, 18 edges
  Article 22: 27 nodes, 16 edges
  Article 23: 3 nodes, 2 edges
  Article 24: 35 nodes, 19 edges
  Article 25: 18 nodes, 11 edges
  Article 26: 38 nodes, 27 edges

In [23]:
# ============================================================================
# DIAGNOSTIC: Check KG Structure
# ============================================================================

# Check what's actually in the first KG
print("Checking structure of all_article_kgs...")
print(f"Total KGs: {len(all_article_kgs)}\n")

# Look at first KG
first_kg = all_article_kgs[0]
print("First KG keys:")
print(first_kg.keys())
print("\nFirst KG structure:")
for key, value in first_kg.items():
    print(f"  {key}: {type(value)}")
    if key not in ['graph']:  # Don't print the whole graph
        print(f"    Value: {value}")

print("\n" + "="*80)

# If there's a 'graph' key with NetworkX object, extract its data
if 'graph' in first_kg:
    graph = first_kg['graph']
    print("\nNetworkX Graph found!")
    print(f"  Nodes: {list(graph.nodes())[:10]}")  # First 10 nodes
    print(f"  Number of nodes: {graph.number_of_nodes()}")
    print(f"  Number of edges: {graph.number_of_edges()}")
    
    # Show a few edges
    edges_sample = list(graph.edges(data=True))[:5]
    print(f"\n  Sample edges:")
    for source, target, data in edges_sample:
        print(f"    {source} -> {target}, data: {data}")

Checking structure of all_article_kgs...
Total KGs: 100

First KG keys:
dict_keys(['id', 'text', 'concepts', 'graph', 'entities', 'relations', 'num_nodes', 'num_edges'])

First KG structure:
  id: <class 'int'>
    Value: 0
  text: <class 'str'>
    Value: 1. Health care reform

How could the health care reform legislation that President Barack Obama signed into law on March 23, 2010, not be the #1 story of the year? Whether you are for or against it, the Patient Protection and Affordable Care Act is nothing if not ambitious, and if implemented, it will fundamentally alter how American health care is financed and perhaps delivered. The law is designed to patch holes in the health insurance system and extend coverage to 32 million Americans by 2019 while also reining in health care spending, which now accounts for more than 17% of the country’s gross domestic product. The biggest changes aren’t scheduled to occur until 2014, when most people will be required to have health insurance or 

In [24]:
# ============================================================================
# CONVERT AND SAVE KNOWLEDGE GRAPHS TO JSON (CORRECTED)
# ============================================================================

import json

def convert_kg_to_json_format(kg_dict):
    """Convert KG dict to JSON-serializable format"""
    json_kg = {
        'id': kg_dict['id'],
        'text': kg_dict['text'],
        'concepts': kg_dict['concepts'],
        'entities': kg_dict['entities'],  # This is the nodes list
        'relations': kg_dict['relations'],  # This is the edges list
        'num_nodes': kg_dict['num_nodes'],
        'num_edges': kg_dict['num_edges']
    }
    return json_kg

# Convert all KGs
print("Converting knowledge graphs to JSON format...")
json_serializable_kgs = [convert_kg_to_json_format(kg) for kg in all_article_kgs]

# Save to JSON
output_path = '/kaggle/working/article_knowledge_graphs.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(json_serializable_kgs, f, indent=2, ensure_ascii=False)

print(f"✓ Saved {len(json_serializable_kgs)} KGs to {output_path}")
print(f"  File size: {os.path.getsize(output_path) / (1024 * 1024):.2f} MB")

# Verify
with open(output_path, 'r') as f:
    loaded = json.load(f)
print(f"✓ Verified! Loaded {len(loaded)} KGs")

Converting knowledge graphs to JSON format...
✓ Saved 100 KGs to /kaggle/working/article_knowledge_graphs.json
  File size: 0.55 MB
✓ Verified! Loaded 100 KGs


In [25]:
# ============================================================================
# STEP 3: LOAD DATA AND EXISTING KNOWLEDGE GRAPHS
# ============================================================================

def load_json_data(filepath: str) -> List[Dict]:
    """Load JSON data from file"""
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"✓ Loaded {len(data)} documents from {filepath}")
    return data

def load_knowledge_graphs(filepath: str) -> List[Dict]:
    """Load previously created knowledge graphs"""
    if not os.path.exists(filepath):
        print(f"\n⚠️  WARNING: Knowledge graph file not found at {filepath}")
        return None
    
    with open(filepath, 'r', encoding='utf-8') as f:
        kgs = json.load(f)
    print(f"✓ Loaded {len(kgs)} knowledge graphs from {filepath}")
    return kgs

# Load your data
articles = load_json_data('/kaggle/input/zstikg/MedicalConcept Data-set.json')
knowledge_graphs = load_knowledge_graphs('/kaggle/working/article_knowledge_graphs.json')

# Only classify articles that have KGs
articles_with_kgs = articles[:len(knowledge_graphs)]  # First 100 articles

if knowledge_graphs is None:
    print("\n❌ Cannot proceed without knowledge graphs. Please save them first.")
else:
    print(f"\n✓ Successfully loaded {len(articles)} articles and {len(knowledge_graphs)} KGs")
    print("✓ Ready to proceed with topic classification!")

✓ Loaded 2066 documents from /kaggle/input/zstikg/MedicalConcept Data-set.json
✓ Loaded 100 knowledge graphs from /kaggle/working/article_knowledge_graphs.json

✓ Successfully loaded 2066 articles and 100 KGs
✓ Ready to proceed with topic classification!


In [26]:
# ============================================================================
# STEP 4: EXTRACT ALL UNIQUE CONCEPTS FROM DATASET
# ============================================================================

def extract_unique_concepts(articles: List[Dict]) -> List[str]:
    """
    Extract all unique concepts from the dataset's 'Concept' field.
    
    Args:
        articles: List of article dictionaries
    
    Returns:
        Sorted list of unique concepts
    """
    all_concepts = set()
    
    for article in articles:
        # Get concepts from the Concept field
        if 'Concept' in article and article['Concept']:
            if isinstance(article['Concept'], list):
                all_concepts.update([c.lower().strip() for c in article['Concept'] if c])
            elif isinstance(article['Concept'], str):
                all_concepts.add(article['Concept'].lower().strip())
    
    # Convert to sorted list
    unique_concepts = sorted(list(all_concepts))
    
    print(f"\n{'='*80}")
    print(f"UNIQUE CONCEPTS EXTRACTED FROM DATASET")
    print(f"{'='*80}")
    print(f"Total unique concepts: {len(unique_concepts)}")
    print(f"\nFirst 20 concepts: {unique_concepts[:20]}")
    print(f"\nLast 20 concepts: {unique_concepts[-20:]}")
    
    return unique_concepts


# Extract all unique concepts from the dataset
all_concepts = extract_unique_concepts(articles)

# Save concept list for reference
with open('/kaggle/working/all_concepts.json', 'w') as f:
    json.dump(all_concepts, f, indent=2)

print(f"\n✓ Saved concept list to /kaggle/working/all_concepts.json")


UNIQUE CONCEPTS EXTRACTED FROM DATASET
Total unique concepts: 18

First 20 concepts: ['addiction', 'alcohol', 'arthritis', 'brain and cognitive health', 'breast cancer', 'cancer', "children's health", 'exercise and fitness', 'headache', 'healthy eating', 'heart health', 'mental health', 'osteoporosis', 'pain management', 'prostate knowledge', 'sleep', 'smoking cessation', "women's health"]

Last 20 concepts: ['addiction', 'alcohol', 'arthritis', 'brain and cognitive health', 'breast cancer', 'cancer', "children's health", 'exercise and fitness', 'headache', 'healthy eating', 'heart health', 'mental health', 'osteoporosis', 'pain management', 'prostate knowledge', 'sleep', 'smoking cessation', "women's health"]

✓ Saved concept list to /kaggle/working/all_concepts.json


In [27]:
# ============================================================================
# STEP 1: IMPORTS
# ============================================================================

import json
import networkx as nx
from typing import List, Dict, Set, Tuple
import dspy
from dspy.signatures import Signature
from dspy import OutputField, InputField
import pandas as pd
import os
from collections import Counter
import numpy as np

print("✓ All imports successful!")

✓ All imports successful!


In [28]:
# ============================================================================
# STEP 2: CONFIGURE LLM
# ============================================================================

os.environ['HUGGINGFACE_API_KEY'] = 'KEY' 
lm = dspy.LM(
    model='huggingface/meta-llama/Llama-3.1-8B-Instruct',
    api_key=os.environ['HUGGINGFACE_API_KEY'],
    max_tokens=12000,
    temperature=0.3
)

dspy.settings.configure(lm=lm)
print("✓ Llama-3.1-8B-Instruct configured successfully!")

✓ Llama-3.1-8B-Instruct configured successfully!


In [29]:
# ============================================================================
# STEP 5: KG FORMATTING FUNCTION
# ============================================================================

def format_kg_for_llm(kg_data: Dict) -> str:
    """Format knowledge graph into readable text for the LLM."""
    if not kg_data or kg_data.get('num_edges', 0) == 0:
        return "No knowledge graph available."
    
    # Format entities
    entities = kg_data.get('entities', [])
    entities_str = ", ".join(entities[:30]) if entities else "None"
    if len(entities) > 30:
        entities_str += f"... ({len(entities) - 30} more)"
    
    # Format relationships
    relations = kg_data.get('relations', [])
    if relations:
        relationships = []
        for relation in relations[:20]:
            if isinstance(relation, (list, tuple)) and len(relation) >= 3:
                source, rel_type, target = relation[0], relation[1], relation[2]
                relationships.append(f"{source} --[{rel_type}]--> {target}")
        relationships_str = "\n".join(relationships)
        if len(relations) > 20:
            relationships_str += f"\n... ({len(relations) - 20} more relationships)"
    else:
        relationships_str = "None"
    
    kg_summary = f"""Knowledge Graph Summary:
Entities: {entities_str}

Key Relationships:
{relationships_str}"""
    
    return kg_summary

print("✓ KG formatting function ready")

✓ KG formatting function ready


In [30]:
# ============================================================================
# STEP 6: DEFINE DSPY SIGNATURE FOR TOPIC CLASSIFICATION
# ============================================================================

class TopicClassification(Signature):
    """Given an article text, its knowledge graph, and a list of possible topics,
    determine which topics (can be multiple) are most relevant to this article.
    Return only topic names that are actually present in the available_topics list."""
    
    article_text: str = InputField(
        desc="The text content of the article"
    )
    knowledge_graph: str = InputField(
        desc="Knowledge graph extracted from the article showing entities and relationships"
    )
    available_topics: str = InputField(
        desc="Comma-separated list of all possible topic/concept names"
    )
    
    predicted_topics: str = OutputField(
        desc="Comma-separated list of relevant topics from the available_topics list. "
             "Only return topics that actually appear in the available_topics list. "
             "If multiple topics apply, list them all. If no topics match well, return 'none'."
    )
    confidence: str = OutputField(
        desc="Confidence level: high, medium, or low"
    )
    reasoning: str = OutputField(
        desc="Brief explanation of why these topics were chosen based on the article and knowledge graph"
    )

print("✓ Topic classification signature defined")

✓ Topic classification signature defined


In [31]:
# ============================================================================
# STEP 7: CREATE TOPIC CLASSIFIER CLASS
# ============================================================================

class ArticleTopicClassifier:
    """Multi-label topic classifier using article text, KG, and available topics."""
    
    def __init__(self, available_topics: List[str]):
        self.available_topics = available_topics
        self.available_topics_str = ", ".join(available_topics)
        self.classifier = dspy.ChainOfThought(TopicClassification)
        print(f"✓ Classifier initialized with {len(available_topics)} possible topics")
    
    def classify_article(self, article_text: str, kg_data: Dict) -> Dict:
        """Classify a single article."""
        kg_summary = format_kg_for_llm(kg_data)
        
        # Truncate article if too long
        max_text_length = 500
        if len(article_text) > max_text_length:
            article_text = article_text[:max_text_length] + "..."
        
        try:
            result = self.classifier(
                article_text=article_text,
                knowledge_graph=kg_summary,
                available_topics=self.available_topics_str
            )
            
            predicted_topics_raw = result.predicted_topics
            
            if predicted_topics_raw.lower() == 'none':
                predicted_topics = []
            else:
                predicted_topics = [
                    t.strip().lower() 
                    for t in predicted_topics_raw.split(',')
                    if t.strip()
                ]
                predicted_topics = [
                    t for t in predicted_topics 
                    if t in [at.lower() for at in self.available_topics]
                ]
            
            return {
                'predicted_topics': predicted_topics,
                'num_topics': len(predicted_topics),
                'confidence': result.confidence,
                'reasoning': result.reasoning
            }
            
        except Exception as e:
            print(f"  Error: {str(e)}")
            return {
                'predicted_topics': [],
                'num_topics': 0,
                'confidence': 'error',
                'reasoning': f'Classification failed: {str(e)}'
            }
    
    def classify_dataset(self, articles: List[Dict], knowledge_graphs: List[Dict],
                        max_articles: int = None) -> List[Dict]:
        """Classify multiple articles."""
        results = []
        kg_dict = {kg['id']: kg for kg in knowledge_graphs}
        
        if max_articles:
            articles = articles[:max_articles]
        
        print(f"\n{'='*80}")
        print(f"CLASSIFYING {len(articles)} ARTICLES")
        print(f"{'='*80}\n")
        
        for idx, article in enumerate(articles):
            print(f"Processing article {idx}...", end=" ")
            
            article_text = article.get('Article Text', '')
            kg_data = kg_dict.get(idx, {'num_edges': 0})
            classification = self.classify_article(article_text, kg_data)
            
            result = {
                'article_id': idx,
                'article_text': article_text[:100] + "...",
                'true_concepts': article.get('Concept', []),
                'predicted_topics': classification['predicted_topics'],
                'num_predicted': classification['num_topics'],
                'confidence': classification['confidence'],
                'reasoning': classification['reasoning']
            }
            
            results.append(result)
            print(f"✓ Found {len(classification['predicted_topics'])} topics")
        
        return results

print("✓ Classifier class defined")

✓ Classifier class defined


In [32]:
# ============================================================================
# STEP 8: ANALYSIS FUNCTIONS
# ============================================================================

def analyze_classification_results(results: List[Dict], available_topics: List[str]) -> Dict:
    """Analyze classification results and generate statistics."""
    from collections import Counter
    
    topic_counts = Counter()
    confidence_dist = Counter()
    
    multi_label_count = 0
    no_label_count = 0
    
    for result in results:
        predicted = result['predicted_topics']
        
        if len(predicted) == 0:
            no_label_count += 1
        elif len(predicted) > 1:
            multi_label_count += 1
        
        for topic in predicted:
            topic_counts[topic] += 1
        
        confidence_dist[result['confidence']] += 1
    
    stats = {
        'total_articles': len(results),
        'articles_with_topics': len(results) - no_label_count,
        'articles_without_topics': no_label_count,
        'multi_label_articles': multi_label_count,
        'avg_topics_per_article': sum(len(r['predicted_topics']) for r in results) / len(results),
        'most_common_topics': topic_counts.most_common(10)
    }
    
    print(f"\n{'='*80}")
    print("CLASSIFICATION RESULTS SUMMARY")
    print(f"{'='*80}")
    print(f"\nTotal articles: {stats['total_articles']}")
    print(f"Articles with topics: {stats['articles_with_topics']} ({stats['articles_with_topics']/stats['total_articles']*100:.1f}%)")
    print(f"Multi-label articles: {stats['multi_label_articles']} ({stats['multi_label_articles']/stats['total_articles']*100:.1f}%)")
    print(f"Average topics per article: {stats['avg_topics_per_article']:.2f}")
    
    print(f"\n{'='*80}")
    print("TOP 10 MOST FREQUENT TOPICS")
    print(f"{'='*80}")
    for topic, count in stats['most_common_topics']:
        print(f"  {topic}: {count} articles ({count/stats['total_articles']*100:.1f}%)")
    
    return stats

def save_results(results: List[Dict], output_path: str):
    """Save classification results to JSON file."""
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    print(f"\n✓ Results saved to {output_path}")

def export_to_csv(results: List[Dict], output_path: str):
    """Export results to CSV for easy analysis."""
    rows = []
    for r in results:
        rows.append({
            'article_id': r['article_id'],
            'article_preview': r['article_text'],
            'true_concepts': '|'.join(r['true_concepts']) if r['true_concepts'] else '',
            'predicted_topics': '|'.join(r['predicted_topics']),
            'num_predicted': r['num_predicted'],
            'confidence': r['confidence'],
            'reasoning': r['reasoning']
        })
    
    df = pd.DataFrame(rows)
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"✓ CSV exported to {output_path}")

print("✓ Analysis functions defined")

✓ Analysis functions defined


In [33]:
# ============================================================================
# RUN CLASSIFICATION ON 100 ARTICLES
# ============================================================================

# Initialize classifier
classifier = ArticleTopicClassifier(all_concepts)

# Classify all 100 articles with KGs
results = classifier.classify_dataset(
    articles=articles,
    knowledge_graphs=knowledge_graphs,
    max_articles=100
)

# Analyze results
stats = analyze_classification_results(results, all_concepts)

# Save results
save_results(results, '/kaggle/working/topic_classification_results.json')
export_to_csv(results, '/kaggle/working/topic_classification_results.csv')

# Show sample results
print(f"\n{'='*80}")
print("SAMPLE RESULTS (First 3)")
print(f"{'='*80}\n")

for result in results[:3]:
    print(f"Article {result['article_id']}:")
    print(f"  Preview: {result['article_text']}")
    print(f"  True: {result['true_concepts']}")
    print(f"  Predicted: {result['predicted_topics']}")
    print(f"  Confidence: {result['confidence']}")
    print()

✓ Classifier initialized with 18 possible topics

CLASSIFYING 100 ARTICLES

Processing article 0... ✓ Found 5 topics
Processing article 1... ✓ Found 2 topics
Processing article 2... ✓ Found 5 topics
Processing article 3... ✓ Found 3 topics
Processing article 4... ✓ Found 2 topics
Processing article 5... ✓ Found 2 topics
Processing article 6... ✓ Found 2 topics
Processing article 7... ✓ Found 3 topics
Processing article 8... ✓ Found 2 topics
Processing article 9... ✓ Found 2 topics
Processing article 10... ✓ Found 5 topics
Processing article 11... ✓ Found 2 topics
Processing article 12... ✓ Found 5 topics
Processing article 13... ✓ Found 3 topics
Processing article 14... ✓ Found 2 topics
Processing article 15... ✓ Found 2 topics
Processing article 16... ✓ Found 2 topics
Processing article 17... ✓ Found 3 topics
Processing article 18... ✓ Found 2 topics
Processing article 19... ✓ Found 2 topics
Processing article 20... 

2025/10/27 22:01:58 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


  Error: litellm.APIError: HuggingfaceException - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}
✓ Found 0 topics
Processing article 21... ✓ Found 1 topics
Processing article 22... ✓ Found 2 topics
Processing article 23... ✓ Found 3 topics
Processing article 24... ✓ Found 2 topics
Processing article 25... ✓ Found 1 topics
Processing article 26... ✓ Found 2 topics
Processing article 27... ✓ Found 2 topics
Processing article 28... ✓ Found 5 topics
Processing article 29... ✓ Found 2 topics
Processing article 30... ✓ Found 1 topics
Processing article 31... ✓ Found 3 topics
Processing article 32... ✓ Found 1 topics
Processing article 33... 

2025/10/27 22:02:49 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


  Error: litellm.APIError: HuggingfaceException - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}
✓ Found 0 topics
Processing article 34... 

...

  Error: litellm.APIError: HuggingfaceException - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}
✓ Found 0 topics
Processing article 97... 

2025/10/27 22:07:21 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


  Error: litellm.APIError: HuggingfaceException - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}
✓ Found 0 topics
Processing article 98... 

2025/10/27 22:07:32 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


✓ Found 0 topics
Processing article 99... ✓ Found 2 topics

CLASSIFICATION RESULTS SUMMARY

Total articles: 100
Articles with topics: 87 (87.0%)
Multi-label articles: 67 (67.0%)
Average topics per article: 1.97

TOP 10 MOST FREQUENT TOPICS
  heart health: 41 articles (41.0%)
  mental health: 31 articles (31.0%)
  healthy eating: 23 articles (23.0%)
  cancer: 19 articles (19.0%)
  exercise and fitness: 14 articles (14.0%)
  prostate knowledge: 14 articles (14.0%)
  women's health: 12 articles (12.0%)
  children's health: 11 articles (11.0%)
  brain and cognitive health: 10 articles (10.0%)
  pain management: 7 articles (7.0%)

✓ Results saved to /kaggle/working/topic_classification_results.json
✓ CSV exported to /kaggle/working/topic_classification_results.csv

SAMPLE RESULTS (First 3)

Article 0:
  Preview: 1. Health care reform

How could the health care reform legislation that President Barack Obama sign...
  True: []
  Predicted: ['heart health', 'healthy eating', 'mental health', "

In [34]:
# ============================================================================
# RUN CLASSIFICATION ON 100 ARTICLES
# ============================================================================

# Initialize classifier
classifier = ArticleTopicClassifier(all_concepts)

# Classify all 100 articles with KGs
results = classifier.classify_dataset(
    articles=articles,
    knowledge_graphs=knowledge_graphs,
    max_articles=100
)

# Analyze results
stats = analyze_classification_results(results, all_concepts)

# Save results
save_results(results, '/kaggle/working/topic_classification_results.json')
export_to_csv(results, '/kaggle/working/topic_classification_results.csv')

# Show sample results
print(f"\n{'='*80}")
print("SAMPLE RESULTS (First 3)")
print(f"{'='*80}\n")

for result in results[:3]:
    print(f"Article {result['article_id']}:")
    print(f"  Preview: {result['article_text']}")
    print(f"  True: {result['true_concepts']}")
    print(f"  Predicted: {result['predicted_topics']}")
    print(f"  Confidence: {result['confidence']}")
    print()

✓ Classifier initialized with 18 possible topics

CLASSIFYING 100 ARTICLES

Processing article 0... ✓ Found 5 topics
Processing article 1... ✓ Found 2 topics
Processing article 2... ✓ Found 5 topics
Processing article 3... ✓ Found 3 topics
Processing article 4... ✓ Found 2 topics
Processing article 5... ✓ Found 2 topics
Processing article 6... ✓ Found 2 topics
Processing article 7... ✓ Found 3 topics
Processing article 8... ✓ Found 2 topics
Processing article 9... ✓ Found 2 topics
Processing article 10... ✓ Found 5 topics
Processing article 11... ✓ Found 2 topics
Processing article 12... ✓ Found 5 topics
Processing article 13... ✓ Found 3 topics
Processing article 14... ✓ Found 2 topics
Processing article 15... ✓ Found 2 topics
Processing article 16... ✓ Found 2 topics
Processing article 17... ✓ Found 3 topics
Processing article 18... ✓ Found 2 topics
Processing article 19... ✓ Found 2 topics
Processing article 20... ✓ Found 3 topics
Processing article 21... ✓ Found 1 topics
Processing

In [35]:
# ============================================================================
# EVALUATION: Compare Predicted vs True Concepts
# ============================================================================

def evaluate_classification(results: List[Dict]) -> Dict:
    """
    Calculate accuracy metrics by comparing predicted topics with true concepts.
    """
    total = len(results)
    exact_match = 0  # All predicted topics match exactly with true concepts
    partial_match = 0  # At least one topic matches
    no_match = 0  # No topics match
    no_true_labels = 0  # Articles with no true concepts
    
    all_precisions = []
    all_recalls = []
    all_f1s = []
    
    correct_predictions = []
    incorrect_predictions = []
    
    for result in results:
        true_set = set([c.lower().strip() for c in result['true_concepts']]) if result['true_concepts'] else set()
        pred_set = set(result['predicted_topics'])
        
        # Skip articles with no true labels
        if len(true_set) == 0:
            no_true_labels += 1
            continue
        
        # Find intersection
        intersection = true_set & pred_set
        
        # Exact match
        if true_set == pred_set and len(true_set) > 0:
            exact_match += 1
            correct_predictions.append(result)
        elif len(intersection) > 0:
            partial_match += 1
        else:
            no_match += 1
            incorrect_predictions.append(result)
        
        # Calculate precision, recall, F1
        if len(pred_set) > 0:
            precision = len(intersection) / len(pred_set)
            all_precisions.append(precision)
        else:
            all_precisions.append(0)
        
        if len(true_set) > 0:
            recall = len(intersection) / len(true_set)
            all_recalls.append(recall)
        else:
            all_recalls.append(0)
        
        if all_precisions[-1] + all_recalls[-1] > 0:
            f1 = 2 * (all_precisions[-1] * all_recalls[-1]) / (all_precisions[-1] + all_recalls[-1])
            all_f1s.append(f1)
        else:
            all_f1s.append(0)
    
    articles_with_true_labels = total - no_true_labels
    
    metrics = {
        'total_articles': total,
        'articles_with_true_labels': articles_with_true_labels,
        'articles_without_true_labels': no_true_labels,
        'exact_matches': exact_match,
        'partial_matches': partial_match,
        'no_matches': no_match,
        'exact_match_rate': exact_match / articles_with_true_labels if articles_with_true_labels > 0 else 0,
        'partial_match_rate': partial_match / articles_with_true_labels if articles_with_true_labels > 0 else 0,
        'avg_precision': np.mean(all_precisions) if all_precisions else 0,
        'avg_recall': np.mean(all_recalls) if all_recalls else 0,
        'avg_f1': np.mean(all_f1s) if all_f1s else 0,
        'correct_examples': correct_predictions[:3],
        'incorrect_examples': incorrect_predictions[:3]
    }
    
    # Print results
    print(f"\n{'='*80}")
    print("EVALUATION METRICS")
    print(f"{'='*80}")
    print(f"\nTotal articles evaluated: {articles_with_true_labels}")
    print(f"Articles without labels (skipped): {no_true_labels}")
    print(f"\n📊 ACCURACY:")
    print(f"  ✓ Exact matches: {exact_match} ({metrics['exact_match_rate']*100:.1f}%)")
    print(f"  ~ Partial matches: {partial_match} ({metrics['partial_match_rate']*100:.1f}%)")
    print(f"  ✗ No matches: {no_match} ({no_match/articles_with_true_labels*100:.1f}%)")
    print(f"\n📈 PERFORMANCE METRICS:")
    print(f"  Precision: {metrics['avg_precision']:.3f} (how many predicted topics were correct)")
    print(f"  Recall: {metrics['avg_recall']:.3f} (how many true topics were found)")
    print(f"  F1 Score: {metrics['avg_f1']:.3f} (overall accuracy)")
    
    return metrics


# Run evaluation
print("\nEvaluating classification accuracy...")
eval_metrics = evaluate_classification(results)

# Show correct examples
print(f"\n{'='*80}")
print("✓ CORRECTLY CLASSIFIED EXAMPLES")
print(f"{'='*80}")
for ex in eval_metrics['correct_examples']:
    print(f"\nArticle {ex['article_id']}:")
    print(f"  True: {ex['true_concepts']}")
    print(f"  Predicted: {ex['predicted_topics']}")
    print(f"  ✓ EXACT MATCH!")

# Show incorrect examples
print(f"\n{'='*80}")
print("✗ INCORRECTLY CLASSIFIED EXAMPLES")
print(f"{'='*80}")
for ex in eval_metrics['incorrect_examples']:
    print(f"\nArticle {ex['article_id']}:")
    print(f"  True: {ex['true_concepts']}")
    print(f"  Predicted: {ex['predicted_topics']}")
    print(f"  Reasoning: {ex['reasoning'][:150]}...")


Evaluating classification accuracy...

EVALUATION METRICS

Total articles evaluated: 76
Articles without labels (skipped): 24

📊 ACCURACY:
  ✓ Exact matches: 38 (50.0%)
  ~ Partial matches: 36 (47.4%)
  ✗ No matches: 2 (2.6%)

📈 PERFORMANCE METRICS:
  Precision: 0.739 (how many predicted topics were correct)
  Recall: 0.932 (how many true topics were found)
  F1 Score: 0.798 (overall accuracy)

✓ CORRECTLY CLASSIFIED EXAMPLES

Article 1:
  True: ['Heart Health', "Women's Health"]
  Predicted: ['heart health', "women's health"]
  ✓ EXACT MATCH!

Article 4:
  True: ['Healthy Eating', 'Heart Health']
  Predicted: ['healthy eating', 'heart health']
  ✓ EXACT MATCH!

Article 5:
  True: ['Exercise and Fitness', 'Heart Health']
  Predicted: ['heart health', 'exercise and fitness']
  ✓ EXACT MATCH!

✗ INCORRECTLY CLASSIFIED EXAMPLES

Article 57:
  True: ['Mental Health']
  Predicted: ['brain and cognitive health']
  Reasoning: The article discusses a study on a three-protein signature that ma

In [36]:
# ============================================================================
# EVALUATION METRICS (Following the Paper's Methodology)
# ============================================================================

def calculate_metrics_paper_style(results: List[Dict]) -> Dict:
    """
    Calculate evaluation metrics following the paper's approach:
    "For each article, the model inferred topic(s) were compared against 
    the list of 'gold' topic(s) to compute the true positive, false positive, 
    and false negative statistics for that article. Then, all such statistics 
    for all the articles in a dataset were aggregated and used to compute 
    the final Precision, Recall, and micro-averaged F1 score."
    
    Reference: Section 5.4 of the paper
    """
    # Aggregate statistics across all articles
    total_tp = 0  # True Positives
    total_fp = 0  # False Positives
    total_fn = 0  # False Negatives
    
    articles_evaluated = 0
    articles_skipped = 0
    
    for result in results:
        # Get true and predicted topics (lowercased and stripped)
        true_topics = set([c.lower().strip() for c in result['true_concepts']]) if result['true_concepts'] else set()
        pred_topics = set([t.lower().strip() for t in result['predicted_topics']])
        
        # Skip articles with no ground truth labels
        if len(true_topics) == 0:
            articles_skipped += 1
            continue
        
        articles_evaluated += 1
        
        # Calculate TP, FP, FN for this article
        tp = len(true_topics & pred_topics)  # Intersection
        fp = len(pred_topics - true_topics)  # Predicted but not true
        fn = len(true_topics - pred_topics)  # True but not predicted
        
        # Aggregate
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
    # Calculate micro-averaged metrics
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'total_tp': total_tp,
        'total_fp': total_fp,
        'total_fn': total_fn,
        'articles_evaluated': articles_evaluated,
        'articles_skipped': articles_skipped
    }
    
    # Print results
    print(f"\n{'='*80}")
    print("EVALUATION METRICS (Paper's Methodology)")
    print(f"{'='*80}")
    print(f"\nArticles evaluated: {articles_evaluated}")
    print(f"Articles skipped (no ground truth): {articles_skipped}")
    print(f"\nAggregated Statistics:")
    print(f"  True Positives (TP): {total_tp}")
    print(f"  False Positives (FP): {total_fp}")
    print(f"  False Negatives (FN): {total_fn}")
    print(f"\n📊 MICRO-AVERAGED METRICS:")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall: {recall:.3f}")
    print(f"  F1 Score: {f1_score:.3f}")
    
    return metrics


# Run evaluation using the paper's methodology
print("\nEvaluating using paper's methodology...")
paper_metrics = calculate_metrics_paper_style(results)

# Compare with their baselines (from Table 6 in the paper)
print(f"\n{'='*80}")
print("COMPARISON WITH PAPER'S BASELINES (Medical Dataset)")
print(f"{'='*80}")
print(f"Your F1 Score: {paper_metrics['f1_score']:.3f}")
print(f"\nPaper's Results for Medical Dataset:")
print(f"  GFLM-S baseline: 0.532")
print(f"  GFLM-W baseline: 0.530")
print(f"  SBERT (best mid encoder): 0.594")
print(f"  ChatGPT-3.5 (best overall): 0.606")


Evaluating using paper's methodology...

EVALUATION METRICS (Paper's Methodology)

Articles evaluated: 76
Articles skipped (no ground truth): 24

Aggregated Statistics:
  True Positives (TP): 106
  False Positives (FP): 58
  False Negatives (FN): 10

📊 MICRO-AVERAGED METRICS:
  Precision: 0.646
  Recall: 0.914
  F1 Score: 0.757

COMPARISON WITH PAPER'S BASELINES (Medical Dataset)
Your F1 Score: 0.757

Paper's Results for Medical Dataset:
  GFLM-S baseline: 0.532
  GFLM-W baseline: 0.530
  SBERT (best mid encoder): 0.594
  ChatGPT-3.5 (best overall): 0.606


In [37]:
# ============================================================================
# DIAGNOSTIC: Check for potential issues
# ============================================================================

def diagnose_results(results: List[Dict], all_concepts: List[str]):
    """Check what might be causing inflated metrics."""
    
    print(f"\n{'='*80}")
    print("DIAGNOSTIC ANALYSIS")
    print(f"{'='*80}")
    
    # Check for direct entity-topic matches
    print("\n1. Checking for data leakage (entity names matching topic names)...")
    
    # Sample a few KGs to see if entities match topics
    if len(knowledge_graphs) > 0:
        sample_kg = knowledge_graphs[0]
        entities = set([e.lower() for e in sample_kg.get('entities', [])])
        topics = set([t.lower() for t in all_concepts])
        
        direct_matches = entities & topics
        if direct_matches:
            print(f"   ⚠️  WARNING: Found {len(direct_matches)} direct entity-topic matches:")
            print(f"   {list(direct_matches)[:10]}")
            print("   This could cause data leakage!")
        else:
            print("   ✓ No direct entity-topic name matches found")
    
    # Check prediction statistics
    print("\n2. Prediction statistics:")
    topics_per_article = [len(r['predicted_topics']) for r in results]
    print(f"   Average predicted topics per article: {np.mean(topics_per_article):.2f}")
    print(f"   Max predicted topics: {max(topics_per_article)}")
    print(f"   Min predicted topics: {min(topics_per_article)}")
    
    # Check how many articles have predictions
    articles_with_predictions = sum(1 for r in results if len(r['predicted_topics']) > 0)
    print(f"   Articles with predictions: {articles_with_predictions}/{len(results)}")
    
    # Check confidence distribution
    print("\n3. Confidence distribution:")
    confidences = [r['confidence'] for r in results]
    from collections import Counter
    conf_dist = Counter(confidences)
    for conf, count in conf_dist.most_common():
        print(f"   {conf}: {count} articles ({count/len(results)*100:.1f}%)")
    
    # Show some examples
    print(f"\n4. Sample predictions (first 5 articles):")
    for i, result in enumerate(results[:5]):
        print(f"\n   Article {i}:")
        print(f"   True: {result['true_concepts']}")
        print(f"   Predicted: {result['predicted_topics']}")
        match = set(result['true_concepts']) & set(result['predicted_topics'])
        print(f"   Matches: {match if match else 'None'}")

# Run diagnostics
diagnose_results(results, all_concepts)

# Then run evaluation
paper_metrics = calculate_metrics_paper_style(results)


DIAGNOSTIC ANALYSIS

1. Checking for data leakage (entity names matching topic names)...
   ⚠️  WARNING: Found 1 direct entity-topic matches:
   ['osteoporosis']
   This could cause data leakage!

2. Prediction statistics:
   Average predicted topics per article: 2.16
   Max predicted topics: 12
   Min predicted topics: 0
   Articles with predictions: 94/100

3. Confidence distribution:
   high: 93 articles (93.0%)
   medium: 4 articles (4.0%)
   medium

The confidence level is medium because the article provides some evidence for the effectiveness of echinacea, but also mentions limitations and potential drawbacks.: 2 articles (2.0%)
   low: 1 articles (1.0%)

4. Sample predictions (first 5 articles):

   Article 0:
   True: []
   Predicted: ['heart health', 'healthy eating', 'mental health', "women's health", "children's health"]
   Matches: None

   Article 1:
   True: ['Heart Health', "Women's Health"]
   Predicted: ['heart health', "women's health"]
   Matches: None

   Article 2